# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 17 2023 2:08PM,259512,19,021661,"Emersa Waterbox, LLC",Released
1,Mar 17 2023 1:59PM,259511,10,U1244,"Uniderm USA, Inc",Released
2,Mar 17 2023 1:57PM,259510,19,021664,"Emersa Waterbox, LLC",Released
3,Mar 17 2023 1:55PM,259509,22,643941,"NBTY Global, Inc.",Released
4,Mar 17 2023 1:45PM,259508,19,021663,"Emersa Waterbox, LLC",Released
5,Mar 17 2023 1:36PM,259506,19,021655,"Emersa Waterbox, LLC",Released
6,Mar 17 2023 1:29PM,259503,19,012660,"Emersa Waterbox, LLC",Released
7,Mar 17 2023 1:20PM,259501,10,9273302,Eywa Pharma Inc.,Released
8,Mar 17 2023 1:20PM,259501,10,9273907,Eywa Pharma Inc.,Released
9,Mar 17 2023 1:20PM,259501,10,9274504,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,259508,Released,1
17,259509,Released,1
18,259510,Released,1
19,259511,Released,1
20,259512,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259508,NaN,NaN,1.0
259509,NaN,NaN,1.0
259510,NaN,NaN,1.0
259511,NaN,NaN,1.0
259512,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259473,11.0,4.0,8.0
259478,20.0,2.0,0.0
259489,0.0,0.0,4.0
259493,0.0,0.0,1.0
259494,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259473,11,4,8
259478,20,2,0
259489,0,0,4
259493,0,0,1
259494,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259473,11,4,8
1,259478,20,2,0
2,259489,0,0,4
3,259493,0,0,1
4,259494,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259473,11,4,8
1,259478,20,2,
2,259489,,,4
3,259493,,,1
4,259494,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 17 2023 2:08PM,259512,19,"Emersa Waterbox, LLC"
1,Mar 17 2023 1:59PM,259511,10,"Uniderm USA, Inc"
2,Mar 17 2023 1:57PM,259510,19,"Emersa Waterbox, LLC"
3,Mar 17 2023 1:55PM,259509,22,"NBTY Global, Inc."
4,Mar 17 2023 1:45PM,259508,19,"Emersa Waterbox, LLC"
5,Mar 17 2023 1:36PM,259506,19,"Emersa Waterbox, LLC"
6,Mar 17 2023 1:29PM,259503,19,"Emersa Waterbox, LLC"
7,Mar 17 2023 1:20PM,259501,10,Eywa Pharma Inc.
12,Mar 17 2023 1:06PM,259499,18,Innova Softgel LLC
13,Mar 17 2023 12:51PM,259497,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 17 2023 2:08PM,259512,19,"Emersa Waterbox, LLC",,,1
1,Mar 17 2023 1:59PM,259511,10,"Uniderm USA, Inc",,,1
2,Mar 17 2023 1:57PM,259510,19,"Emersa Waterbox, LLC",,,1
3,Mar 17 2023 1:55PM,259509,22,"NBTY Global, Inc.",,,1
4,Mar 17 2023 1:45PM,259508,19,"Emersa Waterbox, LLC",,,1
5,Mar 17 2023 1:36PM,259506,19,"Emersa Waterbox, LLC",,,1
6,Mar 17 2023 1:29PM,259503,19,"Emersa Waterbox, LLC",,,1
7,Mar 17 2023 1:20PM,259501,10,Eywa Pharma Inc.,,,5
8,Mar 17 2023 1:06PM,259499,18,Innova Softgel LLC,,,1
9,Mar 17 2023 12:51PM,259497,10,Bio-PRF,,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 2:08PM,259512,19,"Emersa Waterbox, LLC",1,,
1,Mar 17 2023 1:59PM,259511,10,"Uniderm USA, Inc",1,,
2,Mar 17 2023 1:57PM,259510,19,"Emersa Waterbox, LLC",1,,
3,Mar 17 2023 1:55PM,259509,22,"NBTY Global, Inc.",1,,
4,Mar 17 2023 1:45PM,259508,19,"Emersa Waterbox, LLC",1,,
5,Mar 17 2023 1:36PM,259506,19,"Emersa Waterbox, LLC",1,,
6,Mar 17 2023 1:29PM,259503,19,"Emersa Waterbox, LLC",1,,
7,Mar 17 2023 1:20PM,259501,10,Eywa Pharma Inc.,5,,
8,Mar 17 2023 1:06PM,259499,18,Innova Softgel LLC,1,,
9,Mar 17 2023 12:51PM,259497,10,Bio-PRF,7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 2:08PM,259512,19,"Emersa Waterbox, LLC",1,,
1,Mar 17 2023 1:59PM,259511,10,"Uniderm USA, Inc",1,,
2,Mar 17 2023 1:57PM,259510,19,"Emersa Waterbox, LLC",1,,
3,Mar 17 2023 1:55PM,259509,22,"NBTY Global, Inc.",1,,
4,Mar 17 2023 1:45PM,259508,19,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 2:08PM,259512,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Mar 17 2023 1:59PM,259511,10,"Uniderm USA, Inc",1.0,NaN,NaN
2,Mar 17 2023 1:57PM,259510,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,Mar 17 2023 1:55PM,259509,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Mar 17 2023 1:45PM,259508,19,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 17 2023 2:08PM,259512,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Mar 17 2023 1:59PM,259511,10,"Uniderm USA, Inc",1.0,0.0,0.0
2,Mar 17 2023 1:57PM,259510,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,Mar 17 2023 1:55PM,259509,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Mar 17 2023 1:45PM,259508,19,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2075980,28.0,0.0,0.0
15,259473,8.0,4.0,11.0
18,259499,1.0,0.0,0.0
19,1557017,5.0,2.0,20.0
22,519003,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2075980,28.0,0.0,0.0
1,15,259473,8.0,4.0,11.0
2,18,259499,1.0,0.0,0.0
3,19,1557017,5.0,2.0,20.0
4,22,519003,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,28.0,0.0,0.0
1,15,8.0,4.0,11.0
2,18,1.0,0.0,0.0
3,19,5.0,2.0,20.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,28.0
1,15,Released,8.0
2,18,Released,1.0
3,19,Released,5.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,22
Status,,,,,
Completed,0.0,11.0,0.0,20.0,0.0
Executing,0.0,4.0,0.0,2.0,0.0
Released,28.0,8.0,1.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,22
0,Completed,0.0,11.0,0.0,20.0,0.0
1,Executing,0.0,4.0,0.0,2.0,0.0
2,Released,28.0,8.0,1.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,22
0,Completed,0.0,11.0,0.0,20.0,0.0
1,Executing,0.0,4.0,0.0,2.0,0.0
2,Released,28.0,8.0,1.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()